In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import arch
import ivolatility as ivol
import os

# Script description: https://www.ivolatility.com/api/docs#section/Python-Examples/Example-No.-4:-Auto-ARIMA-GARCH-Forecasting

ivolApiKey = os.getenv('API_KEY')

getMarketData = ivol.setMethod('/equities/eod/stock-prices')
marketData = getMarketData(symbol='SPX', from_='2020-01-01', to='2022-06-30')

marketData['date'] = pd.to_datetime(marketData['date'])
marketData = marketData.sort_values(by='date')
marketData = marketData.reset_index(drop=True)
close_prices = marketData['close'] * 0.1  #rescaling close_prices by 0.1 to mitigate DataScaleWarning

model = pm.auto_arima(close_prices, start_p=1, start_q=1,
test='adf',
max_p=3,
max_q=3,
m=12,
start_P=0,
seasonal=False,
d=None,
D=1,
trace=True,
error_action='ignore',
suppress_warnings=True,
stepwise=False)

#Print ARIMA results
print(model.summary())

model.plot_diagnostics(figsize=(7,5))
plt.show()

#Get ARIMA residuals
arima_residuals = model.arima_res_.resid

#fit a GARCH(1,1) model on the residuals of the ARIMA model
garch = arch.arch_model(arima_residuals, p=1, q=1)
garch_fitted = garch.fit()

#Print GARCH results
print(garch_fitted.summary())
garch_fitted.plot(annualize='D')
plt.show()

#Forecast
n_periods = 90
index_of_fc = np.arange(len(close_prices), len(close_prices)+n_periods)

#Use ARIMA to predict mu
predicted_mu, confint = model.predict(n_periods=n_periods, return_conf_int=True)

#Use GARCH to predict the residual
garch_forecast = garch_fitted.forecast(horizon=n_periods)
predicted_et = garch_forecast.mean['h.01'].iloc[-1]
prediction = predicted_mu + predicted_et

fitted_series = pd.Series(prediction, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

#Print prediction
print(prediction)

#Plot
plt.plot(close_prices)
plt.plot(fitted_series, color='darkgreen')
plt.fill_between(lower_series.index,
lower_series,
upper_series,
color='k', alpha=.15)

plt.title('Forecast of SPX')
plt.show()